# Introduction to RAG with Amazon Bedrock

In [11]:
# Import necessary libraries
import json
import os
import sys
import boto3
from langchain_community.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [12]:
# Initialize AWS Bedrock client
bedrock=boto3.client(service_name="bedrock-runtime")
bedrock_embeddings=BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",client=bedrock)

In [13]:
## Data ingestion
def data_ingestion():
    loader=PyPDFDirectoryLoader("data")
    documents=loader.load()
    
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=10000,
                                                 chunk_overlap=1000)
    
    docs=text_splitter.split_documents(documents)
    return docs

In [14]:
## Vector Embedding and vector store
def get_vector_store(docs):
    vectorstore_faiss=FAISS.from_documents(
        docs,
        bedrock_embeddings
    )
    vectorstore_faiss.save_local("faiss_index")

In [36]:
def get_llama3_llm():
    ##create the Anthropic Model
    llm=Bedrock(model_id="meta.llama3-8b-instruct-v1:0",client=bedrock,
                model_kwargs={'max_gen_len':512})
    
    return llm

In [16]:
prompt_template = """

Human: Use the following pieces of context to provide a 
concise answer to the question at the end but use atleast summarize with 
250 words with detailed explainations. If you don't know the answer, 
just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

Question: {question}

Assistant:"""

In [17]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])


In [18]:
def get_response_llm(llm,vectorstore_faiss,query):
    qa = RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",
                                     retriever=vectorstore_faiss.as_retriever(search_type="similarity", 
                                                                              search_kwargs={"k": 3}),
                                     return_source_documents=True,chain_type_kwargs={"prompt": PROMPT})
    answer=qa({"query":query})
    return answer['result']

In [28]:
def main():
    user_question = "summary please"
    docs = data_ingestion()
    get_vector_store(docs)
    faiss_index = FAISS.load_local("faiss_index", bedrock_embeddings,allow_dangerous_deserialization=True)
    llm=get_llama3_llm()
    #faiss_index = get_vector_store(docs)
    print(get_response_llm(llm,faiss_index,user_question))

In [37]:
if __name__ == "__main__":
    main()

 

Based on the provided context, I will summarize the main points and provide a concise answer to the question. 

The provided context is a collection of research papers and articles related to neural network-based video compression. The papers discuss various techniques and models for compressing video data using neural networks, including autoencoders, transformers, and quantization methods. The context also includes information about the UVG and MCL-JVC datasets, which are used for evaluating the performance of video compression algorithms. 

The question is not explicitly stated in the provided context, but based on the content, it appears to be related to the topic of neural network-based video compression. Therefore, I will provide a summary of the main points and highlight some of the key findings and techniques discussed in the context. 

The main points discussed in the context are:

1. The use of neural networks for video compression, including autoencoders and transformers.